<br>
<center>

# Assignments for Week 3

</center>

<div style="text-align: right; width: 15%; margin-left: auto;">
    
Jon Cannaday <br>
DSC 550 <br>
3/28/2024  

</div>


## Libraries

In [2]:
import pandas as pd
from textblob import TextBlob
from sklearn.metrics import accuracy_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## Part 1: Using the TextBlob Sentiment Analyzer
<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

   - ### Import the movie review data as a data frame and ensure that the data is loaded properly.


In [3]:
path = r'C:\Users\Owner\OneDrive\Desktop\DataScience\2024_Spring_DataMining\labeledTrainData.tsv'
data = pd.read_csv(path, sep='\t')


print(type(data)) # Data type
print()
print(data.head()) # Data check

<class 'pandas.core.frame.DataFrame'>

       id  sentiment                                             review
0  5814_8          1  With all this stuff going down at the moment w...
1  2381_9          1  \The Classic War of the Worlds\" by Timothy Hi...
2  7759_3          0  The film starts with a manager (Nicholas Bell)...
3  3630_4          0  It must be assumed that those who praised this...
4  9495_8          1  Superbly trashy and wondrously unpretentious 8...


In [4]:
print(data)

            id  sentiment                                             review
0       5814_8          1  With all this stuff going down at the moment w...
1       2381_9          1  \The Classic War of the Worlds\" by Timothy Hi...
2       7759_3          0  The film starts with a manager (Nicholas Bell)...
3       3630_4          0  It must be assumed that those who praised this...
4       9495_8          1  Superbly trashy and wondrously unpretentious 8...
...        ...        ...                                                ...
24995   3453_3          0  It seems like more consideration has gone into...
24996   5064_1          0  I don't believe they made this film. Completel...
24997  10905_3          0  Guy is a loser. Can't get girls, needs to buil...
24998  10194_3          0  This 30 minute documentary Buñuel made in the ...
24999   8478_8          1  I saw this movie as a child and it broke my he...

[25000 rows x 3 columns]


   - ### How many of each positive and negative reviews are there?


In [57]:
positive = data[data['sentiment'] == 1] # Boolean filter where condition it true
negative = data[data['sentiment'] == 0]

print("Positive reviews: ", len(positive)) # Number of elements
print("Negative reviews: ", len(negative))

Positive reviews:  12500
Negative reviews:  12500


   - ### Use TextBlob to classify each movie review as positive or negative. Assume that a polarity score greater than or equal to zero is a positive sentiment and less than 0 is a negative sentiment.

In [58]:
# Create prediction column using Polarity which quantifies the sentiment returns binary 
data['predicted_sentiment'] = data['review'].apply(lambda text: 1 if TextBlob(text).sentiment.polarity >= 0 else 0)

print()
print(data['predicted_sentiment'])


0        1
1        1
2        0
3        1
4        0
        ..
24995    1
24996    1
24997    1
24998    1
24999    1
Name: predicted_sentiment, Length: 25000, dtype: int64


   - ### Check the accuracy of this model. Is this model better than random guessing?


In [59]:
accuracy_textblob = accuracy_score(data['sentiment'], data['predicted_sentiment']) # Accuracy statment
print()
print("Model accuracy : ", accuracy_textblob) # Results
print()
print("Which would be better than guessing, we have a 50/50 shot at guessing correctly.")


Model accuracy :  0.68524

Which would be better than guessing, we have a 50/50 shot at guessing correctly.


   - ### For up to five points extra credit, use another prebuilt text sentiment analyzer, e.g., VADER, and repeat steps (3) and (4).

In [60]:
vader_analyzer = SentimentIntensityAnalyzer() # Initiate 

# Create a prediction Column using the polarity scores
data['predicted_sentiment_vader'] = data['review'].apply(lambda text: 1 if vader_analyzer.polarity_scores(text)['compound'] >= 0 else 0)

# Calculate the accuracy
accuracy_vader = accuracy_score(data['sentiment'], data['predicted_sentiment_vader'])
print()
print("VADER model accuracy : ", accuracy_vader)


VADER model accuracy :  0.69356


## Part 2: Prepping Text for a Custom Model
<br>
<div style="width: 100%; border-bottom: 1px solid black;"></div>

   - ### Convert all text to lowercase letters.


In [61]:
data['review'] = data['review'].str.lower() # Lower case
print()
print(data['review'].head())


0    with all this stuff going down at the moment w...
1    \the classic war of the worlds\" by timothy hi...
2    the film starts with a manager (nicholas bell)...
3    it must be assumed that those who praised this...
4    superbly trashy and wondrously unpretentious 8...
Name: review, dtype: object


   - ### Remove punctuation and special characters from the text.


In [62]:
# not in logic to remove the punctuation
data['review'] = data['review'].apply(lambda text: ''.join(char for char in text if char not in string.punctuation))
print()
print(data['review'].head())


0    with all this stuff going down at the moment w...
1    the classic war of the worlds by timothy hines...
2    the film starts with a manager nicholas bell g...
3    it must be assumed that those who praised this...
4    superbly trashy and wondrously unpretentious 8...
Name: review, dtype: object


   - ### Remove stop words.


In [63]:
# not in logic to remove the stop words
stop_words = set(stopwords.words('english'))
data['review'] = data['review'].apply(lambda text: ' '.join(word for word in text.split() if word not in stop_words))

print()
print(data['review'].head())


0    stuff going moment mj ive started listening mu...
1    classic war worlds timothy hines entertaining ...
2    film starts manager nicholas bell giving welco...
3    must assumed praised film greatest filmed oper...
4    superbly trashy wondrously unpretentious 80s e...
Name: review, dtype: object


   - ### Apply NLTK’s PorterStemmer.


In [64]:
# Initiates PorterStemmer
stemmer = PorterStemmer()

# create a review column and split 
data['review'] = data['review'].apply(lambda text: ' '.join(stemmer.stem(word) for word in text.split()))

print()
print(data['review'].head())


0    stuff go moment mj ive start listen music watc...
1    classic war world timothi hine entertain film ...
2    film start manag nichola bell give welcom inve...
3    must assum prais film greatest film opera ever...
4    superbl trashi wondrous unpretenti 80 exploit ...
Name: review, dtype: object


   - ### Create a bag-of-words matrix from your stemmed text (output from (4)), where each row is a word-count vector for a single movie review (see sections 5.3 & 6.8 in the Machine Learning with Python Cookbook). Display the dimensions of your bag-of-words matrix. The number of rows in this matrix should be the same as the number of rows in your original data frame.


In [65]:
# initiates port stemmer
stemmer = PorterStemmer()

# Custom column 
data['review'] = data['review'].apply(lambda text: ' '.join(stemmer.stem(word) for word in text.split()))

# Matrix
vectorizer = CountVectorizer()
bag_of_words_matrix = vectorizer.fit_transform(data['review'])

# Matrix dimensions
print("Dimensions of the bag-of-words matrix:", bag_of_words_matrix.shape)

Dimensions of the bag-of-words matrix: (25000, 91587)


   - ### Create a term frequency-inverse document frequency (tf-idf) matrix from your stemmed text, for your movie reviews (see section 6.9 in the Machine Learning with Python Cookbook). Display the dimensions of your tf-idf matrix. These dimensions should be the same as your bag-of-words matrix.

In [66]:
stemmer = PorterStemmer()

# Custom colum that passes each split work idto the stemmer
data['review'] = data['review'].apply(lambda text: ' '.join(stemmer.stem(word) for word in text.split()))

# tf-idf matrix
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['review'])

# tf-idf Matrix dimensions
print("Dimensions of the tf-idf matrix:", tfidf_matrix.shape)

Dimensions of the tf-idf matrix: (25000, 91473)
